In [1]:
import re
import getpass
import gmaps
import requests
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from urllib.parse import urljoin
from IPython.display import clear_output, display
from parse import parse
import yaml
import time
import os
import sys
from fabric import Connection

#adding prognos tools
sys.path.insert(0, "/home/jose-luis/Envs/niva_prognos/PROGNOS/")
from prognos_tools.encrypt import decryptString
from prognos_tools.Basin_fabric2 import Basin


gmapsToken=b'gAAAAABctxJp5wE73qK6U5VieCi0WXNnNex4KxLZHutsESa8fW9v43lLa1Ag0qsxKFjXXo9MBAdvUpqJPW-QmCE0gH_Opf9g4xAG1VaI2WarO_xDZg44VLMCHkd_6O8ofgp8u4VuFBMr'
key = getpass.getpass('Password: ')
apiKey = decryptString(gmapsToken,key)
gmaps.configure(api_key=apiKey)
del key,apiKey

#Creating folder to store temporary files and downloaded data
saveFolder = './DownloadedData/'

args = [saveFolder,]
!!rm -rf {args[0]} && mkdir -p {args[0]}

Password:  ·······


[]

# Getting list of openDAP links with high-resolution (1km2) temperature and precipitation data

In [2]:
getList = False

baseURL = 'http://thredds.met.no/thredds/catalog/ngcd/version_19.03/catalog.html'
varsToGet = ['TX','TN','TG','RR']
model = 'type1'
listFolder = os.path.abspath('./' + model + 'Lists') #Folder where list of links will be saved

only_a_tags = SoupStrainer("a", href=True)

def getSoup(url,re_str):
    request=requests.get(url)
    soup=BeautifulSoup(request.content,'lxml',parse_only=only_a_tags)
    link_soup=soup.find_all('a',text=re.compile(re_str))
    links=[]
    for i in link_soup:
        links.append(urljoin(url,i['href']))
    return links

def getFileList(var):
    allLinks=[]
    for var_link in getSoup(baseURL,var):
        for model_link in getSoup(var_link,model):
            for year_link in getSoup(model_link,'^[0-9]{4}/$'):
                print('Processing {}'.format(year_link))
                for month_link in getSoup(year_link,'^[0-9]{2}/$'):
                    for day_link in getSoup(month_link,'\\.nc$'):
                        for opendap_link in getSoup(day_link,'^/thredds/dodsC/'):
                            allLinks.append(re.sub(r'\.html$', '', opendap_link))
    return allLinks
    

if getList:
    links = []
    !! rm -rf listFolder && mkdir listFolder
    for i in varsToGet:
        #Getting all links for variable
        links.append(getFileList(i))
        #Saving paths to file
        with open(os.path.join(listFolder , i + '_' + model + '.txt'), 'w') as f:
            for item in links[-1]:
                f.write("%s\n" % item)

## Querying extent of the basin
### Creating a VM with the geodatabase on it

In [2]:
#Testing connection to google cloud
cloudInfo = {'project': 'nivacatchment',
             'zone': 'europe-west3-a',
             'instanceType' : "n1-standard-2",
             'instanceName' : "prognosin",
             'username' : "jose-luis",
             'keyDir' : './'
             }
basin=Basin('/home/jose-luis/Envs/gce_framework/code/keys/nivacatchment.json',cloudInfo)
display(basin.properties)

info = basin.get('projectInfo')
display('Can now talk to project {}'.format(info['name']))


#Setting ssh firewall for project
inOffice = True
if not inOffice:
    u = miniupnpc.UPnP()
    u.discoverdelay = 200
    u.discover()
    u.selectigd()
    externalIP= u.externalipaddress()
    basin.setSSHPort(externalIP,inOffice=False)
else:
    basin.setSSHPort()

#Actually instantiating the machine
ip=basin.instantiate(wait=True) #wait until the vm is actually up and running
display(ip)

#Allowing local connections on the VM without password by modifying the pg_hba.conf file
with Connection(host=basin.properties['ip'],
                user=basin.properties['username'],
                connect_kwargs={"key_filename": basin.properties['keyFile'],}
                ) as c:
    c.sudo('''find /etc -name pg_hba.conf -exec sed -i 's|\(^host \+all \+all \+::1/128 \+\).*|\\1trust|g' {} \; ''')
    c.sudo("service postgresql restart")


#Defining custom queries to geodatabase. Note that we forwarded the port on the VM to the
#local 5432 port so as to be able to talk to postgresql through the local port
def query(query):
    with Connection(host=basin.properties['ip'],
                    user=basin.properties['username'],
                    connect_kwargs={"key_filename": basin.properties['keyFile']}).forward_local(5432):
        db = psycopg2.connect(host='localhost', port=5432, database='geonorway')
        cursor = db.cursor()
        cursor.execute(query)
        return(cursor.fetchall()) 

#Setting up the connection    
basin.setConnection()

[]

KeyboardInterrupt: 

### Getting basins extent

In [4]:
#Schema were processing will be done
schema = 'metno'

#Putting ljb data as yaml
import pandas as pd
import yaml
ljb = pd.read_csv('~/Downloads/Stream_Monitoring_Locations_forJLG.csv',encoding='iso-8859-1',header=1)
coords = [{'station': {'station_name': i['Prøvested:'].strip(),
                        'station_id': j,
                        'longitude': i['°E'],
                        'latitude': i['°N'],
                        'epsg': 4326,
                        'buffer': 20000
                       }
          } for j,i in ljb.iterrows()]

for i in coords:
    if i['station']['station_name'] == 'Dypingbekken':
        i['station']['longitude'] = 10.017462
        i['station']['latitude'] = 59.849623
    elif i['station']['station_name'] == 'Sandungsbekken':
        i['station']['longitude'] = 10.048416
        i['station']['latitude'] = 59.854848
    elif i['station']['station_name'] == 'Langevannsbekken':
        i['station']['longitude'] = 10.060717
        i['station']['latitude'] = 59.853656
    elif i['station']['station_name'] == 'Guritjernsbekken':
        i['station']['longitude'] = 10.060792
        i['station']['latitude'] = 59.873171

with open('ljb.yaml','w') as f:
    f.write(yaml.dump(coords,allow_unicode=True))

#Getting basin outlets from yaml file
stationsFile = 'ljb.yaml'
stations = yaml.safe_load(open(stationsFile))
display(stations)

#Actually processing the basin
geoResultsFolder='./geoResults'
basin.getBasinLayers(stationsFile,schema,saveFolder=geoResultsFolder)
    
# # Now we delete the basinGenerator vm
#basinGenerator.kill()

[{'station': {'buffer': 20000,
   'epsg': 4326,
   'latitude': 59.88929,
   'longitude': 10.12331,
   'station_id': 0,
   'station_name': 'Rotua ved Kluftetangene'}},
 {'station': {'buffer': 20000,
   'epsg': 4326,
   'latitude': 59.881780000000006,
   'longitude': 10.0896,
   'station_id': 1,
   'station_name': 'Elgtjernsbekken'}},
 {'station': {'buffer': 20000,
   'epsg': 4326,
   'latitude': 59.873171,
   'longitude': 10.060792,
   'station_id': 2,
   'station_name': 'Guritjernsbekken'}},
 {'station': {'buffer': 20000,
   'epsg': 4326,
   'latitude': 59.86846,
   'longitude': 10.03501,
   'station_id': 3,
   'station_name': 'Jøssjøbekken'}},
 {'station': {'buffer': 20000,
   'epsg': 4326,
   'latitude': 59.854848,
   'longitude': 10.048416,
   'station_id': 4,
   'station_name': 'Sandungsbekken'}},
 {'station': {'buffer': 20000,
   'epsg': 4326,
   'latitude': 59.849623,
   'longitude': 10.017462,
   'station_id': 5,
   'station_name': 'Dypingbekken'}},
 {'station': {'buffer': 20000

'Directory should be created'

Gonna create the getBasin.sh file
./geoResults/getBasin.sh


NOTICE:  table "stations" does not exist, skipping


 initializestations 
--------------------
 
(1 row)

 addstations 
-------------
 
(1 row)



...

SET
SET
          dropgeometrycolumn           
---------------------------------------
 metno.dummy.geom effectively removed.
(1 row)

DROP TABLE
BEGIN
CREATE TABLE
ALTER TABLE
                addgeometrycolumn                
-------------------------------------------------
 metno.dummy.geom SRID:3035 TYPE:POLYGON DIMS:2 
(1 row)

INSERT 0 1
COMMIT
ANALYZE
INSERT 0 1
9
Initializing... 
Done (postgis major version: 2).
Output shape: Point
Dumping: X [1 rows].
Gage Watershed version 5.3.8
Input file PG:dbname=geonorway schema=metno table=flow column=rast where='station_id=9' mode=2 has projected coordinate system.


This run may take on the order of 1 minutes to complete.
This estimate is very approximate. 
Run time is highly uncertain as it depends on the complexity of the input data 
and speed and memory of the computer. This estimate is based on our testing on 
a dual quad core Dell Xeon E5405 2.0GHz PC with 16GB RAM.


Size: 2
Read time: 0.469838
Compute time: 0.075835
Write time: 0.069007
Total time: 0.614680
Creating output basin9.shp of format ESRI Shapefile.
0...10...20...30...40...50...60...70...80...90...100 - done.


Shapefile type: Polygon
Postgis type: POLYGON[2]


SET
SET
          dropgeometrycolumn           
---------------------------------------
 metno.dummy.geom effectively removed.
(1 row)

DROP TABLE
BEGIN
CREATE TABLE
ALTER TABLE
                addgeometrycolumn                
-------------------------------------------------
 metno.dummy.geom SRID:3035 TYPE:POLYGON DIMS:2 
(1 row)

INSERT 0 1
COMMIT
ANALYZE
INSERT 0 1
DROP TABLE
COPY 10
COPY 1
COPY 10


### Displaying basins

In [5]:
with open(os.path.join(geoResultsFolder,'gmaps.txt'), 'r') as f:
    lines = f.read().splitlines()
    
fig = gmaps.figure()
fig.add_layer(gmaps.geojson_layer(yaml.safe_load(lines[0])))

lines[2:] = [i.split(';') for i in lines[2:]]
outlets = [{"name": i[0], "area": i[3]} for i in lines[2:]]
locations = [(float(i[2]),float(i[1])) for i in lines[2:]]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Area</dt><dd>{area} km<sup>2</sup></dd>
</dl>
"""                                                
outlet_info = [info_box_template.format(**outlet) for outlet in outlets]                                                 
marker_layer = gmaps.marker_layer(locations, info_box_content=outlet_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))

# Getting data from metno thredds servers
## Creating a virtual machine with the fimex utility

In [ ]:
#And now we can go about creating the vm with fimex on it
# instanceName = 'fimexin'
# username = 'jose-luis'
# keyDir = 'fimex'
# region = 'europe-north1-c'
# machineType = 'n1-standard-32'

machineInfo={}
#Vm characteristics
machineInfo['instance'] = 'fimexin'
machineInfo['username'] = 'jose-luis'
machineInfo['image'] = 'fimex'
machineInfo['imageProject'] = 'nivacatchment'
machineInfo['keyDir'] = '/home/jose-luis/.ssh/fimex'
machineInfo['region'] = 'europe-north1-c'
machineInfo['machineType'] = 'n1-standard-32'
machineInfo['keyFile'] = F'{os.path.join(machineInfo["keyDir"],machineInfo["username"])}'
machineInfo['pubKeyFile'] = F'{machineInfo["keyFile"] + ".pub"}'

fimexGenerator = basin.Basin(machineInfo)
fimexGenerator.instantiate(fabfile='fabfile_fimex.py')
print("The ip of the google cloud instance is {}".format(fimexGenerator.ip))

#Need to wait until the instance is up and running. There must be a better way.
time.sleep(10)

#Testing connection to instance
fimexGenerator.callPopen('gcloud compute ssh {username}@{instance} --zone {region}'.format(
**fimexGenerator.machineInfo))

## Creating cfg file for download

In [ ]:
cfgTemplate='''[extract]
reduceToBoundingBox.south={south}
reduceToBoundingBox.north={north}
reduceToBoundingBox.west={west}
reduceToBoundingBox.east={east}
selectVariables={variable}
'''

## Getting all variables for all basins using fimex

In [ ]:
#Getting all boundaries to download:
with open(os.path.join(geoResultsFolder,'box.txt'), 'r') as f:
    lines = f.read().splitlines()
boxes = [parse('BOX({west} {south},{east} {north})',i.split(';')[1]) for i in lines]
names =  [re.split(';|,|\s',i)[0] for i in lines]
# import os
for i in varsToGet:
    #Getting only dates from 2010 to 2016
    args = [os.path.join(listFolder, 
                         i + '_' + model
                        ), 
            "2014", "2015"
           ]
    !!cat {args[0]}.txt | grep -E `seq -s \| {args[1]} {args[2]}` > {args[0]}_mini.txt
#     'head -n 40 {0}.txt > {0}_mini.txt'.format(i + '_' + model)
    currentDict = boxes[0].named
    currentDict['variable'] = i
    with open('box.cfg','w') as f:
        f.write(cfgTemplate.format(**currentDict))
    for j in names:
        fimexGenerator.getNcDataForBox(os.path.join(listFolder, i + '_' + model) + '_mini.txt',
                                  'box.cfg',
                                  i + '_' + j + '.nc'
                                  )   